# Generate and evaluate images in Amazon Bedrock with Amazon Titan Image Generator

Requirements for the Implementations:
1. Amazon Bedrock Model
2. Amazon Nova Canvas
3. Anthropic Claude 3.5 Sonnet
4. Amazon S3
5. Amazon IAM User Secret key and Access key
6. Amazon Cloudwatch



![alt text](image.png)

## WorkFlow or the execution flow:

Users or applications submit a prompt as an API request.

The prompt and parameters are passed to Amazon Bedrock using an inference API called by the Lambda function.

Amazon Bedrock generates a high-quality image based on the prompt with Amazon Nova Canvas.
The Lambda function sends the image bytes and the original prompt to Anthropic’s Claude 3.5 Sonnet on 

Amazon Bedrock.
Anthropic’s Claude 3.5 Sonnet evaluates the generated image against the original prompt.
The Lambda function saves the image to an Amazon Simple Storage Service (Amazon S3) bucket and generates a 

pre-signed URL.
The pre-signed URL and the evaluation are returned as an API response in JSON format.

### Changes need to done in the CloudFormation stack creation:
A name for the new S3 bucket that will receive the images (for example, image-gen-your-initials)

The name of an existing S3 bucket where access logs will be stored.

A token that you will use to authenticate your API (a string of your choice)

![alt text](image-1.png)

## Test the Model in the AWS

![alt text](image-2.png)

## Result

![alt text](image-3.png)

# Test the model in the Visual Studio code using the Code

In [2]:
import boto3
import json
import base64
import io
from PIL import Image

# Set up the Amazon Bedrock client
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1"
)

# Define the model ID
model_id = "amazon.nova-canvas-v1:0"

# Prepare the input prompt
prompt = "Generate the image on indian flag on the moon"

# Create the request payload
body = json.dumps({
        "taskType": "TEXT_IMAGE",
        "textToImageParams": {
            "text": prompt
        },
        "imageGenerationConfig": {
            "numberOfImages": 1,
            "height": 1024,
            "width": 1024,
            "cfgScale": 8.0,
            "seed": 0
        }
    })

accept = "application/json"
content_type = "application/json"

# Invoke the Amazon Bedrock model
response = bedrock_client.invoke_model(
    modelId=model_id,
    body=body,
    accept=accept, 
    contentType=content_type
)

# Process the response
result = json.loads(response["body"].read())

base64_image = result.get("images")[0]
base64_bytes = base64_image.encode('ascii')
image_bytes = base64.b64decode(base64_bytes)

image = Image.open(io.BytesIO(image_bytes))
image.show()

# Generated Image:

![/](image-5.png)